In [22]:
import random as rn

In [23]:
def btoi(x):
    return int.from_bytes(x, 'little')

In [24]:
## First, some preliminaries that will be needed.

import hashlib

def sha512(s):
    return hashlib.sha512(s).digest()

# Base field Z_p
p = 2**255 - 19

def modp_inv(x):
    return pow(x, p-2, p)

# Curve constant
d = -121665 * modp_inv(121666) % p

# Group order
q = 2**252 + 27742317777372353535851937790883648493

def sha512_modq(s):
    hsh = sha512(s)
    print("HSH: ", hsh.hex())
    return int.from_bytes(hsh, "little") % q

## Then follows functions to perform point operations.

# Points are represented as tuples (X, Y, Z, T) of extended
# coordinates, with x = X/Z, y = Y/Z, x*y = T/Z

def point_inv(X):
    return (0 - X[0]) % p, X[1], X[2], ((0 - X[3]) % p)

def point_add(P, Q):
    A, B = (P[1]-P[0]) * (Q[1]-Q[0]) % p, (P[1]+P[0]) * (Q[1]+Q[0]) % p
    C, D = 2 * P[3] * Q[3] * d % p, 2 * P[2] * Q[2] % p
    E, F, G, H = B-A % p, D-C % p, D+C % p, B+A % p
    return (E*F % p, G*H % p, F*G % p, E*H % p)

def point_sub(P, Q):
    return point_add(P, point_inv(Q))

def point_dub(X):
    A = X[0]**2 % p
    B = X[1]**2 % p
    C = 2*(X[2]**2) % p
    H = A+B % p
    E = H-(X[0]+X[1])**2 % p
    G = A-B % p
    F = C+G % p
    X3 = E*F % p
    Y3 = G*H % p
    T3 = E*H % p
    Z3 = F*G % p
    return (X3, Y3, Z3, T3)

def swap(P, Q):
    return Q, P

# Computes Q = s * Q
def point_mul(s, P):
    Q = (0, 1, 1, 0)  # Neutral element
    for i in range(256, -1, -1):
        if s & (1 << i):
            P, Q = swap(P, Q)
        P = point_add(Q, P)
        Q = point_dub(Q)
        if s & (1 << i):
            P, Q = swap(P, Q)
    return Q

def point_equal(P, Q):
    # x1 / z1 == x2 / z2  <==>  x1 * z2 == x2 * z1
    if (P[0] * Q[2] - Q[0] * P[2]) % p != 0:
        return False
    if (P[1] * Q[2] - Q[1] * P[2]) % p != 0:
        return False
    return True

## Now follows functions for point compression.

# Square root of -1
modp_sqrt_m1 = pow(2, (p-1) // 4, p)

# Compute corresponding x-coordinate, with low bit corresponding to
# sign, or return None on failure
def recover_x(y, sign):
    if y >= p:
        return None
    x2 = (y*y-1) * modp_inv(d*y*y+1)
    if x2 == 0:
        if sign:
            return None
        else:
            return 0

    # Compute square root of x2
    x = pow(x2, (p+3) // 8, p)
    if (x*x - x2) % p != 0:
        x = x * modp_sqrt_m1 % p
    if (x*x - x2) % p != 0:
        return None

    if (x & 1) != sign:
        x = p - x
    return x

# Base point
g_y = 4 * modp_inv(5) % p
g_x = recover_x(g_y, 0)
G = (g_x, g_y, 1, g_x * g_y % p)

def point_compress(P):
    zinv = modp_inv(P[2])
    x = P[0] * zinv % p
    y = P[1] * zinv % p
    return int.to_bytes(y | ((x & 1) << 255), 32, "little")

def point_decompress(s):
    if len(s) != 32:
        raise Exception("Invalid input length for decompression")
    y = int.from_bytes(s, "little")
    sign = y >> 255
    y &= (1 << 255) - 1

    x = recover_x(y, sign)
    if x is None:
        return None
    else:
        return (x, y, 1, x*y % p)

def point_print(X):
    zinv = modp_inv(X[2])
    x = X[0] * zinv % p
    y = X[1] * zinv % p
    print(hex(x), hex(y))

def point_print_full(X):
    print(hex(X[0]), hex(X[1]), hex(X[2]), hex(X[3]))

## These are functions for manipulating the private key.

def secret_expand(secret):
    if len(secret) != 32:
        raise Exception("Bad size of private key")
    h = sha512(secret)
    a = int.from_bytes(h[:32], "little")
    a &= (1 << 254) - 8
    a |= (1 << 254)
    return (a, h[32:])

def secret_to_public(secret):
    (a, dummy) = secret_expand(secret)
    return point_compress(point_mul(a, G))

## The signature function works as below.

def sign(secret, msg):
    a, prefix = secret_expand(secret)
    A = point_compress(point_mul(a, G))
    r = sha512_modq(prefix + msg)
    R = point_mul(r, G)
    Rs = point_compress(R)
    h = sha512_modq(Rs + A + msg)
    s = (r + h * a) % q
    return Rs + int.to_bytes(s, 32, "little")

## And finally the verification function.

def verify(public, msg, signature):
    if len(public) != 32:
        raise Exception("Bad public key length")
    if len(signature) != 64:
        Exception("Bad signature length")
    A = point_decompress(public)
    point_print_full(A)
    if not A:
        return False
    Rs = signature[:32]
    R = point_decompress(Rs)
    if not R:
        return False
    s = int.from_bytes(signature[32:], "little")
    print("S:", hex(s))
    if s >= q: return False
    toh = Rs + public + msg
    print("TOH:", toh.hex())
    h = sha512_modq(Rs + public + msg)
    print("h:", hex(h))
    sB = point_mul(s, G)
    hA = point_mul(h, A)
    point_print_full(sB)
    point_print_full(hA)
    xxx = point_sub(sB, hA)
    point_print(xxx)
    sBmhA = point_compress(xxx)
    print("")
    print("R   :", Rs.hex())
    print("RES : ", sBmhA.hex())
    print(Rs == sBmhA)
    return point_equal(sB, point_add(R, hA))

In [25]:
msg = rn.randint(0, 2**512-1).to_bytes(64, 'little')
secret = rn.randint(0, 2**256-1).to_bytes(32, 'little')

In [26]:
msg = "8f13b3a29344b73d22e681e9faeb3fedb88c94f7504f8f2ac2f17a09fc33a1f6b83275219d83def87aa1f74537c8819db6759c80fff5b4c42aa09b663c5304d9"
msg = binascii.unhexlify(msg)

In [27]:
signature = sign(secret, msg)

print("MSG  :", msg.hex())
print("SIGN :", signature.hex())

public = secret_to_public(secret)
print("PUB  :", public.hex())
print(" -- ")
verify(public, msg, signature)


HSH:  44e5e1e8dade8b5c1f5f2c93b4abe14a6091b5eb82199e3b98ef7e065608e14ae84cd6f1418c1ad5bc8a3f8d9b048e194a053acb761902495f5ef63ba35048eb
HSH:  2544fbed7fbe0fa515cf22c7adbe5a13b881b0e3168b91d33ca6c6a3c7592ac2aec2eb6d27c91512c63b887ee8ebceadc8e08a651a05b3778a04d25be656b56a
MSG  : 8f13b3a29344b73d22e681e9faeb3fedb88c94f7504f8f2ac2f17a09fc33a1f6b83275219d83def87aa1f74537c8819db6759c80fff5b4c42aa09b663c5304d9
SIGN : 2ef9ff1d7926588de9c68104492034a8a8edab57686d95729de313fc70a8623a86031e5bffd2b8fa2b5daf20a09dae43994d209d24042a34ba17cc6cea8ce40f
PUB  : f13c21fd271db83863eab2d4d9a9b503fe745dcb15da3ef5a607a27f7478bbd1
 -- 
0x7fe74ef9bbeaf084d55aa88d401362255214f59d46722a08ca018bd0f2eeecbf 0x51bb78747fa207a6f53eda15cb5d74fe03b5a9d9d4b2ea6338b81d27fd213cf1 0x1 0x68196826ae9137f4263a1e653861101d8843d29cff89574bc3eb57ab2a6d2853
S: 0xfe48cea6ccc17ba342a04249d204d9943ae9da020af5d2bfab8d2ff5b1e0386
TOH: 2ef9ff1d7926588de9c68104492034a8a8edab57686d95729de313fc70a8623af13c21fd271db83863eab2d4d9a9b503fe745d

True